In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import torch
from transformers import Blip2Processor, Blip2ForConditionalGeneration, AutoProcessor, Blip2ForImageTextRetrieval
from datasets import COCODataset
from awq.quantizer import Blip2ForConditionalGenerationAWQQuantizer

import time

In [3]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
device

device(type='cuda')

## AWQ Blip-2 Caption Generation

In [4]:
# SETUP
model_name = "Salesforce/blip2-opt-2.7b"
model = Blip2ForConditionalGeneration.from_pretrained(model_name)
model.to(device)

processor = Blip2Processor.from_pretrained(model_name)

# NOTE: set paths as appropriate
coco_dataset = COCODataset(ann_file='/nfshomes/vla/project_dirs/low-bit-vision/datasets/cocow/annotations/captions_val2017.json',
                           img_dir='/nfshomes/vla/project_dirs/low-bit-vision/datasets/cocow/images/val2017')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


/fs/nexus-scratch/vla/micromamba/envs/MMQ/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
# Apply AWQ
quantizer = Blip2ForConditionalGenerationAWQQuantizer(model, device, processor, coco_dataset)

start = time.time()
quantizer.quantize()
print(f'Quantization time: {time.time() - start:.2f} seconds')

Expanding inputs for image tokens in BLIP-2 should be done in processing. Please follow instruction here (https://gist.github.com/zucchini-nlp/e9f20b054fa322f84ac9311d9ab67042) to update your BLIP-2 model. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.
Quantizing llm_layers: 100%|██████████| 32/32 [05:27<00:00, 10.25s/it]

Quantization time: 660.58 seconds


## AWQ Blip-2 Info-Retrieval